## Import dataset do GIT

In [1]:
import os
import shutil
import zipfile
import git

# Define paths
repo_url = "https://github.com/vitor10102002/Dataset2rotulos.git"
download_path = "C:/content/Dataset2rotulos"
zip_file_path = "C:/content/Dataset2rotulos/Imagens_Resized.zip"
extract_path = "C:/content"

# Clone the repository
if os.path.exists(download_path):
    try:
        shutil.rmtree(download_path)  # Remove the existing folder if any
    except:
        print("")

git.Repo.clone_from(repo_url, download_path)

# Check if the file exists in the cloned repo
if os.path.exists(zip_file_path):
    print("Found Imagens_Resized.zip")
else:
    print("Imagens_Resized.zip not found in the repository.")

# Optionally, unzip the file
if os.path.exists(zip_file_path):
    with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
        zip_ref.extractall(extract_path)
    print(f"Extracted {zip_file_path} to {extract_path}")

Found Imagens_Resized.zip
Extracted C:/content/Dataset2rotulos/Imagens_Resized.zip to C:/content


## Pré-processamento das imagens com o rótulo 1

In [2]:
import cv2
def preprocess_image(image):
    """
    Aplicar pré-processamentos nas imagens:

    """

    resized_image = cv2.resize(image, (224, 224))

    return resized_image

In [3]:
import cv2
import os

# Diretório das imagens originais e processadas
image_folder = "C:\\content\\out_resized"
output_folder = "C:\\content\\processadas\\rotulo_M1"

# Criar pasta de saída se não existir
os.makedirs(output_folder, exist_ok=True)

# Listar arquivos na pasta e filtrar apenas os que começam com "M1_"
image_files = [f for f in os.listdir(image_folder) if f.startswith("M1_") and f.lower().endswith(('.jpg', '.png', '.jpeg'))]

print(f"Encontradas {len(image_files)} imagens M1 para processamento...")

for filename in image_files:
    image_path = os.path.join(image_folder, filename)

    # Ler a imagem
    image = cv2.imread(image_path)
    if image is None:
        print(f"Erro ao carregar {filename}, pulando...")
        continue

    # 🔹 Pré-processamento
    processed_image = preprocess_image(image)

    # Criar um novo nome para evitar sobrescrita
    new_filename = f"processada_{filename}"
    save_path = os.path.join(output_folder, new_filename)

    # Salvar a imagem processada
    cv2.imwrite(save_path, processed_image)

print("✅ Processamento concluído!")


Encontradas 105 imagens M1 para processamento...
✅ Processamento concluído!


## Anomaly Lib

In [2]:
# Import the datamodule
from anomalib.data import Folder
from anomalib.data.utils import TestSplitMode

# Create the datamodule
datamodule = Folder(
    name="Rotulos_marca_M1",
    root="C:\\content\\processadas",
    normal_dir="rotulo_M1",
    test_split_mode=TestSplitMode.SYNTHETIC,
    task="classification",
)

# Setup the datamodule
datamodule.setup()

INFO:anomalib.data.base.datamodule:No normal test images found. Sampling from training set using a split ratio of 0.20


In [3]:
# Import the model and engine
from anomalib.models import Patchcore
from anomalib.engine import Engine

# Create the model and engine
model = Patchcore()
engine = Engine(task="classification")

# Train a Patchcore model on the given datamodule
engine.train(datamodule=datamodule, model=model)

INFO:anomalib.models.components.base.anomaly_module:Initializing Patchcore model.
INFO:timm.models._builder:Loading pretrained weights from Hugging Face hub (timm/wide_resnet50_2.racm_in1k)
INFO:timm.models._hub:[timm/wide_resnet50_2.racm_in1k] Safe alternative available for 'pytorch_model.bin' (as 'model.safetensors'). Loading weights using safetensors.
INFO:timm.models._builder:Missing keys (fc.weight, fc.bias) discovered while loading pretrained weights. This is expected if model is being adapted.
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
INFO:anomalib.data.base.datamodule:No normal test images found. Sampling from training set using a split ratio of 0.20
c:\Users\vitor\anaconda3\envs\Anomaly_m1\lib\site-packages\lightning\pytorch\core\optimizer.py:183: `LightningModule.configure_optimizers` returned `None`, this fit will run with no optimizer

  | Name                  | Type                     | Params | Mode 
-

Training: |          | 0/? [00:00<?, ?it/s]

c:\Users\vitor\anaconda3\envs\Anomaly_m1\lib\site-packages\lightning\pytorch\loops\optimization\automatic.py:134: `training_step` returned `None`. If this was on purpose, ignore this warning...


Validation: |          | 0/? [00:00<?, ?it/s]

INFO:anomalib.models.image.patchcore.lightning_model:Aggregating the embedding extracted from the training set.
INFO:anomalib.models.image.patchcore.lightning_model:Applying core-set subsampling to get the embedding.









: 